In [3]:
import pandas as pd
from string import ascii_uppercase as alpha
import pickle
from bs4 import BeautifulSoup
import requests

In [2]:
all_tables=pd.read_html('https://web.archive.org/web/20221002180521/https://en.wikipedia.org/wiki/2022_FIFA_World_Cup')

In [3]:
dict_tables={}
for letter,i in zip(alpha,range(11,67,7)):
    df=all_tables[i]
    df.rename(columns={df.columns[1]:'Team'},inplace=True)
    dict_tables[f"Cluster:{letter}"]=df

Starting to filter the page

In [2]:
years=[1930,1934,1938,1950,1954,1958,1962,1966,1970,1974,1978,1982,1986,1990,1994,1998,2002,2006,2010,2014,2018]

making a request

In [6]:
web='https://en.wikipedia.org/wiki/2014_FIFA_World_Cup'
response=requests.get(web)

Loading text with Beatifulsoup

In [7]:
content=response.text
soup=BeautifulSoup(content,'lxml')

Filtering rows on matches

In [8]:
matches=soup.findAll('div',class_='fotballbox')

Filtering Matches

In [ ]:
for match in matches:
    match.findAll('th',class_='fhome').get_text()